# Sistema de Gestión de Empleados con Estructuras de Datos

## Materia: Estructuras de Datos

Este notebook implementa un sistema completo de base de datos de empleados utilizando estructuras de datos fundamentales:

- **Árboles Binarios de Búsqueda (BST)** para índices ordenados
- **Tablas Hash** para índices de acceso rápido
- **Listas Enlazadas** para almacenamiento de registros

### Características:
- Tabla de 4 columnas: nombre, edad, sueldo, cargo
- Índices configurables en cualquier columna
- Carga de datos desde archivo CSV
- Entrada manual de datos
- Búsquedas eficientes usando índices

## 1. Importar Librerías

In [ ]:
import csv
import io
from typing import Any, List, Dict, Optional
from dataclasses import dataclass
from enum import Enum
import time

## 2. Estructuras de Datos Fundamentales

### 2.1 Nodo de Lista Enlazada

In [ ]:
@dataclass
class Employee:
    """Clase que representa un empleado (registro de la tabla)"""
    nombre: str
    edad: int
    sueldo: float
    cargo: str
    
    def __repr__(self):
        return f"Employee({self.nombre}, {self.edad}, ${self.sueldo:,.2f}, {self.cargo})"
    
    def to_dict(self):
        return {
            'nombre': self.nombre,
            'edad': self.edad,
            'sueldo': self.sueldo,
            'cargo': self.cargo
        }


class LinkedListNode:
    """Nodo para lista enlazada simple"""
    def __init__(self, data: Employee):
        self.data = data
        self.next = None


class LinkedList:
    """Lista enlazada simple para almacenar empleados"""
    def __init__(self):
        self.head = None
        self.size = 0
    
    def append(self, employee: Employee):
        """Agregar empleado al final de la lista"""
        new_node = LinkedListNode(employee)
        
        if not self.head:
            self.head = new_node
        else:
            current = self.head
            while current.next:
                current = current.next
            current.next = new_node
        
        self.size += 1
        return self.size - 1  # Retorna índice del nuevo elemento
    
    def get(self, index: int) -> Optional[Employee]:
        """Obtener empleado por índice"""
        if index < 0 or index >= self.size:
            return None
        
        current = self.head
        for _ in range(index):
            current = current.next
        
        return current.data
    
    def to_list(self) -> List[Employee]:
        """Convertir lista enlazada a lista de Python"""
        result = []
        current = self.head
        while current:
            result.append(current.data)
            current = current.next
        return result
    
    def __len__(self):
        return self.size


print("✓ Estructura de Lista Enlazada implementada")

### 2.2 Árbol Binario de Búsqueda (BST)

In [ ]:
class BSTNode:
    """Nodo de Árbol Binario de Búsqueda"""
    def __init__(self, key, indices=None):
        self.key = key  # Valor de la columna (ej: edad, nombre, etc.)
        self.indices = indices if indices else []  # Lista de índices de empleados con este valor
        self.left = None
        self.right = None
        self.height = 1  # Para balanceo AVL (opcional)


class BinarySearchTree:
    """Árbol Binario de Búsqueda para índices"""
    def __init__(self):
        self.root = None
        self.node_count = 0
    
    def insert(self, key, index: int):
        """Insertar un valor con su índice en el BST"""
        if self.root is None:
            self.root = BSTNode(key, [index])
            self.node_count += 1
        else:
            self._insert_recursive(self.root, key, index)
    
    def _insert_recursive(self, node: BSTNode, key, index: int):
        """Inserción recursiva en BST"""
        if key == node.key:
            # Valor ya existe, agregar índice a la lista
            node.indices.append(index)
        elif key < node.key:
            if node.left is None:
                node.left = BSTNode(key, [index])
                self.node_count += 1
            else:
                self._insert_recursive(node.left, key, index)
        else:
            if node.right is None:
                node.right = BSTNode(key, [index])
                self.node_count += 1
            else:
                self._insert_recursive(node.right, key, index)
    
    def search(self, key) -> List[int]:
        """Buscar un valor en el BST y retornar lista de índices"""
        node = self._search_recursive(self.root, key)
        return node.indices if node else []
    
    def _search_recursive(self, node: Optional[BSTNode], key) -> Optional[BSTNode]:
        """Búsqueda recursiva en BST"""
        if node is None or node.key == key:
            return node
        
        if key < node.key:
            return self._search_recursive(node.left, key)
        else:
            return self._search_recursive(node.right, key)
    
    def range_search(self, min_key, max_key) -> List[int]:
        """Buscar valores en un rango [min_key, max_key]"""
        result = []
        self._range_search_recursive(self.root, min_key, max_key, result)
        return result
    
    def _range_search_recursive(self, node: Optional[BSTNode], min_key, max_key, result: List[int]):
        """Búsqueda de rango recursiva en BST"""
        if node is None:
            return
        
        # Si el valor está en el rango, agregarlo
        if min_key <= node.key <= max_key:
            result.extend(node.indices)
        
        # Buscar en subárbol izquierdo si es necesario
        if min_key < node.key:
            self._range_search_recursive(node.left, min_key, max_key, result)
        
        # Buscar en subárbol derecho si es necesario
        if max_key > node.key:
            self._range_search_recursive(node.right, min_key, max_key, result)
    
    def inorder_traversal(self) -> List[tuple]:
        """Recorrido inorden del árbol (retorna pares key, indices)"""
        result = []
        self._inorder_recursive(self.root, result)
        return result
    
    def _inorder_recursive(self, node: Optional[BSTNode], result: List[tuple]):
        """Recorrido inorden recursivo"""
        if node:
            self._inorder_recursive(node.left, result)
            result.append((node.key, len(node.indices)))
            self._inorder_recursive(node.right, result)
    
    def get_height(self, node: Optional[BSTNode] = None) -> int:
        """Obtener altura del árbol"""
        if node is None:
            node = self.root
        if node is None:
            return 0
        return 1 + max(self.get_height(node.left), self.get_height(node.right))


print("✓ Estructura de Árbol Binario de Búsqueda implementada")

### 2.3 Tabla Hash con Encadenamiento

In [ ]:
class HashTableNode:
    """Nodo para manejar colisiones mediante encadenamiento"""
    def __init__(self, key, indices=None):
        self.key = key
        self.indices = indices if indices else []
        self.next = None


class HashTable:
    """Tabla Hash con encadenamiento para manejar colisiones"""
    def __init__(self, size=100):
        self.size = size
        self.table = [None] * size
        self.count = 0  # Número de claves únicas
    
    def _hash(self, key) -> int:
        """Función hash"""
        if isinstance(key, str):
            # Hash para strings
            hash_value = 0
            for char in key:
                hash_value = (hash_value * 31 + ord(char)) % self.size
            return hash_value
        elif isinstance(key, (int, float)):
            # Hash para números
            return int(key) % self.size
        else:
            # Hash genérico
            return hash(key) % self.size
    
    def insert(self, key, index: int):
        """Insertar un par clave-índice en la tabla hash"""
        hash_index = self._hash(key)
        
        if self.table[hash_index] is None:
            # No hay colisión
            self.table[hash_index] = HashTableNode(key, [index])
            self.count += 1
        else:
            # Manejar colisión mediante encadenamiento
            current = self.table[hash_index]
            
            # Buscar si la clave ya existe
            while current:
                if current.key == key:
                    current.indices.append(index)
                    return
                if current.next is None:
                    break
                current = current.next
            
            # Agregar nuevo nodo al final de la cadena
            current.next = HashTableNode(key, [index])
            self.count += 1
    
    def search(self, key) -> List[int]:
        """Buscar una clave en la tabla hash"""
        hash_index = self._hash(key)
        current = self.table[hash_index]
        
        while current:
            if current.key == key:
                return current.indices
            current = current.next
        
        return []
    
    def get_all_keys(self) -> List[tuple]:
        """Obtener todas las claves y sus conteos"""
        result = []
        for bucket in self.table:
            current = bucket
            while current:
                result.append((current.key, len(current.indices)))
                current = current.next
        return result
    
    def get_load_factor(self) -> float:
        """Calcular factor de carga"""
        return self.count / self.size
    
    def get_collision_stats(self) -> Dict:
        """Obtener estadísticas de colisiones"""
        used_buckets = 0
        max_chain_length = 0
        total_chain_length = 0
        
        for bucket in self.table:
            if bucket:
                used_buckets += 1
                chain_length = 0
                current = bucket
                while current:
                    chain_length += 1
                    current = current.next
                max_chain_length = max(max_chain_length, chain_length)
                total_chain_length += chain_length
        
        avg_chain_length = total_chain_length / used_buckets if used_buckets > 0 else 0
        
        return {
            'used_buckets': used_buckets,
            'max_chain_length': max_chain_length,
            'avg_chain_length': avg_chain_length,
            'load_factor': self.get_load_factor()
        }


print("✓ Estructura de Tabla Hash implementada")

## 3. Clase Principal: EmployeeTable

### Tabla de empleados con soporte para índices usando las estructuras de datos implementadas

In [ ]:
class IndexType(Enum):
    """Tipo de estructura de datos para el índice"""
    BST = "bst"  # Árbol Binario de Búsqueda
    HASH = "hash"  # Tabla Hash


class EmployeeTable:
    """Tabla de empleados con soporte para índices usando estructuras de datos"""
    
    def __init__(self):
        self.employees = LinkedList()  # Almacenamiento principal usando lista enlazada
        self.indices = {}  # {columna: (tipo_indice, estructura_datos)}
    
    def add_employee(self, nombre: str, edad: int, sueldo: float, cargo: str) -> int:
        """Agregar empleado a la tabla"""
        if cargo not in ['empleado', 'jefe', 'propietario']:
            raise ValueError(f"Cargo inválido: {cargo}. Debe ser: empleado, jefe, o propietario")
        
        employee = Employee(nombre, edad, sueldo, cargo)
        index = self.employees.append(employee)
        
        # Actualizar todos los índices existentes
        for column, (index_type, structure) in self.indices.items():
            value = getattr(employee, column)
            structure.insert(value, index)
        
        return index
    
    def create_index(self, column: str, index_type: IndexType = IndexType.HASH):
        """Crear índice en una columna específica"""
        if column not in ['nombre', 'edad', 'sueldo', 'cargo']:
            raise ValueError(f"Columna inválida: {column}")
        
        if column in self.indices:
            print(f"⚠ La columna '{column}' ya tiene un índice. Reconstruyendo...")
        
        # Crear estructura de datos según el tipo
        if index_type == IndexType.BST:
            structure = BinarySearchTree()
        else:  # HASH
            structure = HashTable()
        
        # Construir el índice recorriendo todos los empleados
        all_employees = self.employees.to_list()
        for idx, employee in enumerate(all_employees):
            value = getattr(employee, column)
            structure.insert(value, idx)
        
        self.indices[column] = (index_type, structure)
        
        type_name = "BST" if index_type == IndexType.BST else "Hash Table"
        print(f"✓ Índice tipo {type_name} creado para la columna '{column}'")
        
        # Mostrar estadísticas
        if index_type == IndexType.BST:
            print(f"  - Altura del árbol: {structure.get_height()}")
            print(f"  - Nodos únicos: {structure.node_count}")
        else:
            stats = structure.get_collision_stats()
            print(f"  - Factor de carga: {stats['load_factor']:.2%}")
            print(f"  - Buckets usados: {stats['used_buckets']}/{structure.size}")
    
    def search(self, column: str, value) -> List[Employee]:
        """Buscar empleados por valor en una columna"""
        if column in self.indices:
            # Usar índice
            _, structure = self.indices[column]
            indices = structure.search(value)
            return [self.employees.get(i) for i in indices]
        else:
            # Búsqueda lineal
            result = []
            all_employees = self.employees.to_list()
            for emp in all_employees:
                if getattr(emp, column) == value:
                    result.append(emp)
            return result
    
    def range_search(self, column: str, min_value, max_value) -> List[Employee]:
        """Buscar empleados en un rango de valores (solo funciona con índice BST)"""
        if column in self.indices:
            index_type, structure = self.indices[column]
            if index_type == IndexType.BST:
                indices = structure.range_search(min_value, max_value)
                return [self.employees.get(i) for i in indices]
            else:
                print("⚠ Búsqueda por rango solo soportada con índice BST. Usando búsqueda lineal.")
        
        # Búsqueda lineal
        result = []
        all_employees = self.employees.to_list()
        for emp in all_employees:
            value = getattr(emp, column)
            if min_value <= value <= max_value:
                result.append(emp)
        return result
    
    def load_from_csv(self, csv_content: str):
        """Cargar empleados desde contenido CSV"""
        reader = csv.DictReader(io.StringIO(csv_content))
        count = 0
        
        for row in reader:
            try:
                nombre = row['nombre'].strip()
                edad = int(row['edad'])
                sueldo = float(row['sueldo'])
                cargo = row['cargo'].strip().lower()
                
                self.add_employee(nombre, edad, sueldo, cargo)
                count += 1
            except (KeyError, ValueError) as e:
                print(f"⚠ Error en fila {count + 1}: {e}")
                continue
        
        print(f"✓ {count} empleados cargados desde CSV")
    
    def load_from_file(self, filename: str):
        """Cargar empleados desde archivo CSV"""
        try:
            with open(filename, 'r', encoding='utf-8') as f:
                content = f.read()
            self.load_from_csv(content)
        except FileNotFoundError:
            print(f"✗ Archivo no encontrado: {filename}")
        except Exception as e:
            print(f"✗ Error al cargar archivo: {e}")
    
    def show_all(self):
        """Mostrar todos los empleados"""
        employees = self.employees.to_list()
        
        if not employees:
            print("No hay empleados en la tabla.")
            return
        
        print("\n" + "=" * 75)
        print(f"{'Nombre':<25} {'Edad':<6} {'Sueldo':<15} {'Cargo':<15}")
        print("=" * 75)
        
        for emp in employees:
            print(f"{emp.nombre:<25} {emp.edad:<6} ${emp.sueldo:<14,.2f} {emp.cargo:<15}")
        
        print("=" * 75)
        print(f"Total: {len(employees)} empleados\n")
    
    def show_index_stats(self):
        """Mostrar estadísticas de los índices"""
        if not self.indices:
            print("No hay índices creados.")
            return
        
        print("\n" + "=" * 60)
        print("ESTADÍSTICAS DE ÍNDICES")
        print("=" * 60)
        
        for column, (index_type, structure) in self.indices.items():
            type_name = "BST" if index_type == IndexType.BST else "Hash Table"
            print(f"\nColumna: {column} (Tipo: {type_name})")
            
            if index_type == IndexType.BST:
                print(f"  - Altura: {structure.get_height()}")
                print(f"  - Nodos: {structure.node_count}")
                print(f"  - Valores (primeros 5):")
                for key, count in structure.inorder_traversal()[:5]:
                    print(f"    • {key}: {count} empleado(s)")
            else:
                stats = structure.get_collision_stats()
                print(f"  - Factor de carga: {stats['load_factor']:.2%}")
                print(f"  - Buckets usados: {stats['used_buckets']}/{structure.size}")
                print(f"  - Longitud máxima de cadena: {stats['max_chain_length']}")
                print(f"  - Longitud promedio de cadena: {stats['avg_chain_length']:.2f}")
        
        print("\n" + "=" * 60)
    
    def get_count(self) -> int:
        """Obtener número de empleados"""
        return len(self.employees)


print("✓ Clase EmployeeTable implementada")

## 4. Funciones Auxiliares

In [ ]:
def create_sample_csv():
    """Crear contenido CSV de ejemplo"""
    return """nombre,edad,sueldo,cargo
Juan Pérez,28,45000,empleado
María García,35,75000,jefe
Carlos López,28,48000,empleado
Ana Martínez,42,150000,propietario
Pedro Sánchez,31,52000,empleado
Laura Rodríguez,35,78000,jefe
Miguel Torres,28,46000,empleado
Sofia Ramírez,39,85000,jefe
Diego Flores,25,42000,empleado
Elena Castro,45,200000,propietario
Roberto Jiménez,28,47000,empleado
Carmen Vega,50,180000,propietario
Fernando Díaz,33,55000,empleado
Isabel Moreno,38,82000,jefe
Luis Herrera,29,49000,empleado"""


def save_sample_csv(filename='empleados.csv'):
    """Guardar CSV de ejemplo en archivo"""
    content = create_sample_csv()
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(content)
    print(f"✓ Archivo '{filename}' creado con datos de ejemplo")


def print_results(results: List[Employee], title: str):
    """Imprimir resultados de búsqueda"""
    print(f"\n{title}")
    print("-" * 70)
    if results:
        print(f"Encontrados {len(results)} resultado(s):\n")
        for emp in results:
            print(f"  • {emp.nombre:<25} - {emp.edad} años - ${emp.sueldo:,.2f} - {emp.cargo}")
    else:
        print("No se encontraron resultados.")
    print()


def compare_performance(table: EmployeeTable, column: str, value):
    """Comparar rendimiento con y sin índice"""
    print(f"\n{'='*60}")
    print(f"COMPARACIÓN DE RENDIMIENTO: {column} = {value}")
    print(f"{'='*60}")
    
    # Búsqueda con índice
    if column in table.indices:
        start = time.perf_counter()
        results = table.search(column, value)
        end = time.perf_counter()
        time_with_index = (end - start) * 1000000  # en microsegundos
        print(f"Con índice: {time_with_index:.2f} μs ({len(results)} resultados)")
    
    # Búsqueda lineal (sin índice)
    start = time.perf_counter()
    results_linear = []
    for emp in table.employees.to_list():
        if getattr(emp, column) == value:
            results_linear.append(emp)
    end = time.perf_counter()
    time_linear = (end - start) * 1000000  # en microsegundos
    print(f"Sin índice (lineal): {time_linear:.2f} μs ({len(results_linear)} resultados)")
    
    if column in table.indices and time_linear > 0:
        speedup = time_linear / time_with_index
        print(f"\n⚡ Aceleración: {speedup:.2f}x más rápido con índice")
    
    print(f"{'='*60}\n")


print("✓ Funciones auxiliares definidas")

## 5. Ejemplos de Uso

### 5.1 Crear tabla y agregar empleados manualmente

In [ ]:
# Crear una tabla nueva
tabla = EmployeeTable()

# Agregar empleados manualmente
print("Agregando empleados manualmente...\n")
tabla.add_employee("Juan Pérez", 28, 45000.00, "empleado")
tabla.add_employee("María García", 35, 75000.00, "jefe")
tabla.add_employee("Carlos López", 28, 48000.00, "empleado")
tabla.add_employee("Ana Martínez", 42, 150000.00, "propietario")

print(f"✓ {tabla.get_count()} empleados agregados")
tabla.show_all()

### 5.2 Cargar empleados desde CSV

In [ ]:
# Crear nueva tabla y cargar desde CSV
tabla = EmployeeTable()

# Opción 1: Cargar desde contenido CSV (texto)
print("Cargando empleados desde CSV...\n")
csv_content = create_sample_csv()
tabla.load_from_csv(csv_content)

tabla.show_all()

### 5.3 Crear índice tipo Hash Table en columna 'edad'

In [ ]:
# Crear índice Hash en edad
print("\nCreando índice tipo Hash Table en columna 'edad'...\n")
tabla.create_index('edad', IndexType.HASH)

# Buscar empleados de 28 años
results = tabla.search('edad', 28)
print_results(results, "EMPLEADOS DE 28 AÑOS (usando índice Hash)")

### 5.4 Crear índice tipo BST en columna 'edad' y búsqueda por rango

In [ ]:
# Crear índice BST en edad (reemplaza el anterior)
print("\nCreando índice tipo BST en columna 'edad'...\n")
tabla.create_index('edad', IndexType.BST)

# Buscar empleados de 28 años
results = tabla.search('edad', 28)
print_results(results, "EMPLEADOS DE 28 AÑOS (usando índice BST)")

# Búsqueda por rango (solo con BST)
results_range = tabla.range_search('edad', 30, 40)
print_results(results_range, "EMPLEADOS ENTRE 30 Y 40 AÑOS (búsqueda por rango con BST)")

### 5.5 Crear índice en columna 'cargo'

In [ ]:
# Crear índice Hash en cargo
print("\nCreando índice tipo Hash en columna 'cargo'...\n")
tabla.create_index('cargo', IndexType.HASH)

# Buscar todos los jefes
results = tabla.search('cargo', 'jefe')
print_results(results, "TODOS LOS JEFES (usando índice Hash)")

# Buscar todos los propietarios
results = tabla.search('cargo', 'propietario')
print_results(results, "TODOS LOS PROPIETARIOS (usando índice Hash)")

### 5.6 Crear índice en columna 'nombre'

In [ ]:
# Crear índice Hash en nombre
print("\nCreando índice tipo Hash en columna 'nombre'...\n")
tabla.create_index('nombre', IndexType.HASH)

# Buscar empleado específico
results = tabla.search('nombre', 'María García')
print_results(results, "BÚSQUEDA POR NOMBRE: María García")

### 5.7 Mostrar estadísticas de todos los índices

In [ ]:
# Mostrar estadísticas de índices
tabla.show_index_stats()

### 5.8 Comparación de rendimiento: Con vs Sin índice

In [ ]:
# Comparar rendimiento
compare_performance(tabla, 'edad', 28)
compare_performance(tabla, 'cargo', 'jefe')
compare_performance(tabla, 'nombre', 'María García')

### 5.9 Agregar nuevos empleados y verificar actualización de índices

In [ ]:
# Agregar nuevos empleados
print("\nAgregando nuevos empleados...\n")
tabla.add_employee("Patricia Gómez", 28, 46500.00, "empleado")
tabla.add_employee("Ricardo Núñez", 28, 47500.00, "empleado")

# Los índices se actualizan automáticamente
results = tabla.search('edad', 28)
print_results(results, "EMPLEADOS DE 28 AÑOS (después de agregar nuevos empleados)")

print(f"Total de empleados en la tabla: {tabla.get_count()}")

## 6. Demostración Interactiva

### Ejemplo completo con datos propios

In [ ]:
# Crear tu propio CSV aquí (modifica los datos como quieras)
mi_csv = """nombre,edad,sueldo,cargo
Tu Nombre,25,50000,empleado
Otro Empleado,30,60000,jefe
Más Empleados,35,80000,propietario
"""

# Crear tabla y cargar
mi_tabla = EmployeeTable()
mi_tabla.load_from_csv(mi_csv)
mi_tabla.show_all()

# Crear índices según necesites
mi_tabla.create_index('edad', IndexType.BST)  # o IndexType.HASH
mi_tabla.create_index('cargo', IndexType.HASH)

# Realizar búsquedas
# mi_tabla.search('edad', 25)
# mi_tabla.range_search('edad', 25, 35)

## 7. Cargar desde archivo CSV externo

In [ ]:
# Primero, guardar archivo de ejemplo
save_sample_csv('empleados.csv')

# Crear tabla y cargar desde archivo
tabla_desde_archivo = EmployeeTable()
tabla_desde_archivo.load_from_file('empleados.csv')

# Mostrar datos cargados
tabla_desde_archivo.show_all()

# Crear índices
tabla_desde_archivo.create_index('edad', IndexType.BST)
tabla_desde_archivo.create_index('cargo', IndexType.HASH)

## 8. Análisis de Complejidad

### Complejidad temporal de las operaciones:

| Operación | Sin Índice | Con Hash Table | Con BST |
|-----------|------------|----------------|----------|
| Inserción | O(1) | O(1) promedio | O(log n) promedio, O(n) peor caso |
| Búsqueda exacta | O(n) | O(1) promedio | O(log n) promedio, O(n) peor caso |
| Búsqueda por rango | O(n) | O(n) | O(log n + k) donde k = resultados |
| Recorrido ordenado | O(n log n) | No soportado | O(n) |

### Ventajas y Desventajas:

**Hash Table:**
- ✓ Búsqueda muy rápida O(1)
- ✓ Inserción muy rápida O(1)
- ✗ No mantiene orden
- ✗ No soporta búsqueda por rango eficiente
- ✗ Usa más memoria

**BST:**
- ✓ Mantiene elementos ordenados
- ✓ Soporta búsqueda por rango eficiente
- ✓ Recorrido ordenado O(n)
- ✗ Puede desbalancearse (peor caso O(n))
- ✗ Más lento que Hash para búsqueda exacta

## 9. Conclusiones

Este notebook demuestra:

1. **Implementación de estructuras de datos fundamentales:**
   - Lista enlazada simple
   - Árbol Binario de Búsqueda (BST)
   - Tabla Hash con encadenamiento

2. **Aplicación práctica:** Sistema de base de datos de empleados con índices

3. **Comparación de estructuras:** Hash Table vs BST para diferentes casos de uso

4. **Flexibilidad:** Soporte para carga de datos desde CSV o entrada manual

5. **Análisis de rendimiento:** Comparación empírica de búsquedas con y sin índices

### Recomendaciones de uso:

- Usa **Hash Table** para búsquedas exactas frecuentes (nombre, cargo)
- Usa **BST** para columnas numéricas con búsquedas por rango (edad, sueldo)
- Considera el costo de memoria vs velocidad según tu aplicación